# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 27 2022 8:30AM,247331,10,129622,"Virtue Rx, LLC",Completed
1,Sep 27 2022 8:30AM,247331,10,129644,"Virtue Rx, LLC",Released
2,Sep 27 2022 8:30AM,247331,10,129651,"Virtue Rx, LLC",Released
3,Sep 27 2022 8:30AM,247332,10,129623,"Virtue Rx, LLC",Released
4,Sep 26 2022 4:15PM,247329,16,8576741,Sartorius Lab Products and Service,Released
5,Sep 26 2022 4:15PM,247329,16,8576748,Sartorius Lab Products and Service,Released
6,Sep 26 2022 4:15PM,247329,16,8576758,Sartorius Lab Products and Service,Released
7,Sep 26 2022 4:15PM,247329,16,8576760,Sartorius Lab Products and Service,Released
8,Sep 26 2022 3:59PM,247328,19,8575389,"Omnivium Pharmaceuticals, LLC",Released
9,Sep 26 2022 3:13PM,247311,10,0086006429,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,247328,Released,1
27,247329,Released,4
28,247331,Completed,1
29,247331,Released,2
30,247332,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247311,NaN,NaN,11.0
247328,NaN,NaN,1.0
247329,NaN,NaN,4.0
247331,1.0,NaN,2.0
247332,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247223,0.0,0.0,8.0
247238,0.0,0.0,1.0
247254,0.0,0.0,7.0
247258,0.0,0.0,1.0
247259,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247223,0,0,8
247238,0,0,1
247254,0,0,7
247258,0,0,1
247259,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247223,0,0,8
1,247238,0,0,1
2,247254,0,0,7
3,247258,0,0,1
4,247259,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247223,,,8
1,247238,,,1
2,247254,,,7
3,247258,,,1
4,247259,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC"
3,Sep 27 2022 8:30AM,247332,10,"Virtue Rx, LLC"
4,Sep 26 2022 4:15PM,247329,16,Sartorius Lab Products and Service
8,Sep 26 2022 3:59PM,247328,19,"Omnivium Pharmaceuticals, LLC"
9,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation
20,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd"
37,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation
61,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation
64,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC"
81,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC",1,,2
1,Sep 27 2022 8:30AM,247332,10,"Virtue Rx, LLC",,,1
2,Sep 26 2022 4:15PM,247329,16,Sartorius Lab Products and Service,,,4
3,Sep 26 2022 3:59PM,247328,19,"Omnivium Pharmaceuticals, LLC",,,1
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,,,11
5,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",,,17
6,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,,,24
7,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,,,3
8,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",,,17
9,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC",2,,1
1,Sep 27 2022 8:30AM,247332,10,"Virtue Rx, LLC",1,,
2,Sep 26 2022 4:15PM,247329,16,Sartorius Lab Products and Service,4,,
3,Sep 26 2022 3:59PM,247328,19,"Omnivium Pharmaceuticals, LLC",1,,
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11,,
5,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",17,,
6,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24,,
7,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3,,
8,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17,,
9,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC",2,,1
1,Sep 27 2022 8:30AM,247332,10,"Virtue Rx, LLC",1,,
2,Sep 26 2022 4:15PM,247329,16,Sartorius Lab Products and Service,4,,
3,Sep 26 2022 3:59PM,247328,19,"Omnivium Pharmaceuticals, LLC",1,,
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC",2,NaN,1.0
1,Sep 27 2022 8:30AM,247332,10,"Virtue Rx, LLC",1,NaN,NaN
2,Sep 26 2022 4:15PM,247329,16,Sartorius Lab Products and Service,4,NaN,NaN
3,Sep 26 2022 3:59PM,247328,19,"Omnivium Pharmaceuticals, LLC",1,NaN,NaN
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC",2,0.0,1.0
1,Sep 27 2022 8:30AM,247332,10,"Virtue Rx, LLC",1,0.0,0.0
2,Sep 26 2022 4:15PM,247329,16,Sartorius Lab Products and Service,4,0.0,0.0
3,Sep 26 2022 3:59PM,247328,19,"Omnivium Pharmaceuticals, LLC",1,0.0,0.0
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3214779,83,0.0,2.0
16,741853,7,1.0,0.0
19,1236470,23,4.0,17.0
20,494555,27,12.0,13.0
22,247298,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3214779,83,0.0,2.0
1,16,741853,7,1.0,0.0
2,19,1236470,23,4.0,17.0
3,20,494555,27,12.0,13.0
4,22,247298,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,83,0.0,2.0
1,16,7,1.0,0.0
2,19,23,4.0,17.0
3,20,27,12.0,13.0
4,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,83.0
1,16,Released,7.0
2,19,Released,23.0
3,20,Released,27.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,22
Status,,,,,
Completed,2.0,0.0,17.0,13.0,0.0
Executing,0.0,1.0,4.0,12.0,0.0
Released,83.0,7.0,23.0,27.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,22
0,Completed,2.0,0.0,17.0,13.0,0.0
1,Executing,0.0,1.0,4.0,12.0,0.0
2,Released,83.0,7.0,23.0,27.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,22
0,Completed,2.0,0.0,17.0,13.0,0.0
1,Executing,0.0,1.0,4.0,12.0,0.0
2,Released,83.0,7.0,23.0,27.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()